In [38]:
%load_ext autoreload
%autoreload 2
import torch
from argparse import ArgumentParser
import pytorch_lightning as pl
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint

from datasets import load_metric

from project.data.data import *
from project.models.rnn import *
from project.metrics.metrics import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Preprocess raw data:

In [39]:
#processor = RaceDataProcessor()
#processor.process_data("RACE", "LON")

## Train model

In [40]:
# Parse arguments:
parser = ArgumentParser()
parser = RaceDataModule.add_model_specific_args(parser)
parser = RaceModule.add_model_specific_args(parser)
parser = pl.Trainer.add_argparse_args(parser)
args = parser.parse_args("--data_path LON \
                          --batch_size 32 \
                          --num_workers 4 \
                          --top_p 0.3 \
                          --hidden_size 128 \
                          --learning_rate 1e-5 \
                          --special_tokens [CON] [QUE] [ANS] [DIS] \
                          --gpus 1 \
                          --max_epochs 1 \
                          --check_val_every_n_epoch 1".split())

pl.seed_everything(1234)


"""# Module and data module:
def customed_collate_fn(batch, tokenizer):
    con_token, que_token, ans_token, dis_token = tokenizer.additional_special_tokens

    inputs = []
    targets = []

    for item in batch:
        inputs.append(" ".join([con_token, item["article"], ans_token, item["answer"]]))
        targets.append(" ".join([que_token, item["question"], dis_token, dis_token.join(item["distractors"])]))

    return {
        "inputs": tokenizer(inputs, padding=True, truncation=True, return_tensors="pt"),
        "targets": tokenizer(targets, padding=True, truncation=True, return_tensors="pt"),
    }"""


data_module = RaceDataModule(args) #, customed_collate_fn)
module = RaceModule(args)


# Callbacks:
checkpoint = ModelCheckpoint(
    dirpath="./checkpoint/fx-{epoch:02d}-{val_loss:.7f}",
    monitor="val_loss"
)


# Trainer:
trainer = pl.Trainer.from_argparse_args(
    args,
    checkpoint_callback=checkpoint
)


#trainer.fit(module, data_module)

Global seed set to 1234
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
GPU available: True, used: True
TPU available: None, using: 0 TPU cores


In [41]:
data_module.prepare_data()
data_module.setup()

In [42]:
trainer.test(module, test_dataloaders=data_module.test_dataloader())


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'bleu_1': 0.0,
 'bleu_2': 0.0,
 'bleu_3': 0.0,
 'bleu_4': 0.0,
 'meteor': 0.014367816091954025,
 'rouge_l': 0.0}
--------------------------------------------------------------------------------


[{'bleu_1': 0.0,
  'bleu_2': 0.0,
  'bleu_3': 0.0,
  'bleu_4': 0.0,
  'meteor': 0.014367816091954025,
  'rouge_l': 0.0}]

In [44]:
data_module.tokenizer

PreTrainedTokenizerFast(name_or_path='prajjwal1/bert-tiny', vocab_size=30522, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]', 'additional_special_tokens': ['[CON]', '[QUE]', '[ANS]', '[DIS]']})